import pandas as pd

In [1]:
import pandas as pd

In [2]:
akakiko = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\Akakiko.csv', error_bad_lines=False);
eliniko = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\To Elliniko Ouzomezedopoleio.csv', error_bad_lines=False);
altius = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\Altius Boutique Hotel.csv', error_bad_lines=False);
hilton = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\Hilton Nicosia.csv', error_bad_lines=False);
jamie = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\Jamies Italian.csv', error_bad_lines=False);
#not in a same cluster
tripadv_review = pd.concat([akakiko, eliniko, altius, hilton, jamie])
tripadv_review

,r_points,r_text,place
0,50,"Excellent food, great atmosphere and service.C...",Akakiko
1,40,"Visited Akakiko Nicosia, a few weeks ago after...",Akakiko
2,50,I have a delivery in this restaurant. Have ama...,Akakiko
3,50,We went there for a lunch break the other days...,Akakiko
4,30,A quick meal after a long flight. The dishes w...,Akakiko
...,...,...,...
376,40,We visited the place onnly 3 days after it ope...,Jamies Italian
377,30,I love Jamie's Italian restaurants - they are ...,Jamies Italian
378,30,Jamie's Italian started operating a few days a...,Jamies Italian
379,30,Ordered penne carbonara. Bacon was overdried. ...,Jamies Italian


In [1]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sent = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
sent.polarity_scores(df.iloc[0]['r_text'])

In [5]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer (stop_words="english", ngram_range = (1,3))
vec.fit(tripadv_review.r_text.values)
features = vec.transform(tripadv_review.r_text.values)

In [8]:
from sklearn.cluster import MiniBatchKMeans, KMeans
clust = KMeans(init='k-means++', n_clusters=10, n_init=1884)

In [9]:
clust.fit(features)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=1884, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [10]:
cl_label = clust.predict(features)
tripadv_review['cluster_label']=clust.labels_
tripadv_review

,r_points,r_text,place,cluster_label
0,50,"Excellent food, great atmosphere and service.C...",Akakiko,2
1,40,"Visited Akakiko Nicosia, a few weeks ago after...",Akakiko,3
2,50,I have a delivery in this restaurant. Have ama...,Akakiko,2
3,50,We went there for a lunch break the other days...,Akakiko,3
4,30,A quick meal after a long flight. The dishes w...,Akakiko,3
...,...,...,...,...
376,40,We visited the place onnly 3 days after it ope...,Jamies Italian,7
377,30,I love Jamie's Italian restaurants - they are ...,Jamies Italian,8
378,30,Jamie's Italian started operating a few days a...,Jamies Italian,8
379,30,Ordered penne carbonara. Bacon was overdried. ...,Jamies Italian,0


In [11]:
tripadv_review.to_csv(r'C:\Users\Dell\Desktop\review eliminator\cluster1.csv')

In [32]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [33]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [35]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0      [excellent, food, great, atmosphere, service, ...
1      [visited, akakiko, nicosia, weeks, renovation,...
2      [delivery, restaurant, amazing, food, enjoy, f...
3      [went, lunch, break, days, restaurant, nicely,...
4      [quick, meal, long, flight, dishes, presented,...
                             ...                        
728    [saturday, room, reservation, hilton, park, ni...
729    [business, trips, couple, times, awesome, meal...
730    [like, noises, city, right, place, drink, coff...
731    [hotel, close, educational, institutions, nico...
732    [stayed, hotel, recently, disappointed, servic...
Name: r_text, Length: 1503, dtype: object

In [42]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]]


In [43]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 400,
                                           eval_every = None)

In [44]:
from pprint import pprint

In [45]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.036*"hotel" + 0.017*"staff" + 0.016*"good" + 0.013*"room" + '
  '0.012*"hilton" + 0.011*"great" + 0.010*"service" + 0.009*"breakfast" + '
  '0.009*"food" + 0.008*"rooms"'),
 (1,
  '0.016*"room" + 0.015*"hotel" + 0.011*"staff" + 0.011*"stay" + '
  '0.009*"nicosia" + 0.007*"cyprus" + 0.006*"great" + 0.006*"food" + '
  '0.006*"hilton" + 0.005*"excellent"'),
 (2,
  '0.023*"hotel" + 0.015*"room" + 0.011*"breakfast" + 0.009*"staff" + '
  '0.009*"good" + 0.008*"stay" + 0.008*"rooms" + 0.007*"business" + '
  '0.007*"excellent" + 0.007*"comfortable"'),
 (3,
  '0.040*"good" + 0.033*"food" + 0.023*"service" + 0.013*"restaurant" + '
  '0.011*"hilton" + 0.009*"nice" + 0.008*"great" + 0.007*"place" + '
  '0.007*"park" + 0.007*"time"'),
 (4,
  '0.040*"hotel" + 0.030*"staff" + 0.020*"breakfast" + 0.020*"room" + '
  '0.019*"good" + 0.017*"stay" + 0.015*"pool" + 0.015*"nicosia" + '
  '0.015*"great" + 0.013*"friendly"'),
 (5,
  '0.010*"hilton" + 0.009*"room" + 0.008*"hotel" + 0.007*"staff" + '


In [46]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.094515  0.104920       1        1  28.877869
8      0.233144  0.029277       2        1  16.590191
6     -0.000680  0.110841       3        1  11.797362
0     -0.062293  0.021092       4        1   7.851222
3      0.124775  0.015243       5        1   7.662971
5      0.029130 -0.136801       6        1   6.594869
9     -0.069798  0.009751       7        1   6.490261
1     -0.040515 -0.041383       8        1   5.531054
2     -0.071472 -0.007241       9        1   5.167153
7     -0.047774 -0.105697      10        1   3.437044, topic_info=           Term         Freq        Total Category  logprob  loglift
500       hotel  1191.000000  1191.000000  Default  30.0000  30.0000
6          food   697.000000   697.000000  Default  29.0000  29.0000
19        sushi   210.000000   210.000000  Default  28.0000  28.0000
33         good   839.000000   839.000000  Default  27.0000  27.0000
1402       room   533.000000   533.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
93       served     4.804831    67.127541  Topic10  -5.8973   0.7336
119    friendly     6.591894   365.193878  Topic10  -5.5811  -0.6440
122        nice     6.952694   536.090149  Topic10  -5.5278  -0.9746
4     excellent     6.130193   414.421204  Topic10  -5.6537  -0.8431
41      quality     4.709285   215.991272  Topic10  -5.9174  -0.4551

[820 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
5410      4  0.799837   ability
2711      3  0.768309  accepted
151       1  0.839404    access
151       4  0.033576    access
151       8  0.033576    access
...     ...       ...       ...
404      10  0.028826      year
5922      9  0.845116     youre
4707      6  0.805775      zone
2116      2  0.868081  zucchini
2525     10  0.445910   zuccini

[2178 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 9, 7, 1, 4, 6, 10, 2, 3, 8])